In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Live Exchange

In [ ]:
#https://www.binance.com/userCenter/openOrders.html
exchange = load_exchange(c.BINANCE)

## Ticker 

In [ ]:
base = c.XRP
quote = c.BTC
asset = Asset(base, quote)
ticker = exchange.fetch_ticker(asset)
pprint(ticker)
price = ticker['bid']

# currently not working as intended... still doesnt provide quantity that meets binance requirements
def min_quantity_needed(min_coin_quantity, price):
    return math.ceil(min_coin_quantity / price)

print("\nPRICE of {} : ".format(base) + str(price))
print("quantity >= {} {}".format(min_quantity_needed(c.MIN_BTC, price), base))
print("Minimum BTC cost = quantity * price = {} BTC".format(price*(.001/price)))
print("Rules : Cost must be > 0.001 BTC or 0.01 ETH or 1 BNB or 1 USDT")

## Fetch Orders

In [ ]:
# Get all my orders
asset = Asset(c.ADA, c.ETH)
exchange.fetch_orders(asset)

# Get order by id
# exchange.fetch_order(order.exchange_order_id, asset.symbol)

## Limit Orders

In [ ]:
# LIMIT BUY (Pending - price lower than market so it doesn't fill)
asset = Asset(c.XRP, c.BTC)
order = exchange.create_limit_buy_order(asset, 20.0, price=0.0001)
order

In [ ]:
# Fetch order just created (to verify)
exchange.fetch_order(order.ex_order_id, asset)

In [ ]:
# LIMIT SELL (Pending - price higher than market so it doesn't fill)
asset = Asset(c.XRP, c.BTC)
order = exchange.create_limit_sell_order(asset, 20.0, price=0.0002)
order

## Market Orders

In [ ]:
asset = Asset(c.XRP, c.BTC)
price = exchange.fetch_ticker(asset)['ask']
price

In [ ]:
# Example market buy
order = exchange.create_market_buy_order(asset, quantity=15.)
order

In [ ]:
# MARKET SELL
order = exchange.create_market_sell_order(asset, quantity=15.)
order

## Cancel Order

In [ ]:
# Check price
asset = Asset(c.XRP, c.BTC)
price = exchange.fetch_ticker(asset)['ask']
price

In [ ]:
# Create pending limit order at a price that is much lower than actual price
order = exchange.create_limit_buy_order(asset, 20.0, price=0.0001)
order

In [ ]:
resp = exchange.cancel_order(order.ex_order_id, asset)
resp

In [ ]:
# Fetch LIMIT order just created
order = exchange.fetch_order(resp['orderId'], asset)
order

## Known Exceptions

In [ ]:
# Example insufficient amount * price
order = exchange.create_limit_buy_order(asset, 100000.0, price)

## Other Stuff

In [ ]:
# Example of how to get amount to lots for exchanges
pprint(exchange.client.amount_to_lots(asset.symbol, quantity))


In [ ]:
# Example order book data
pprint(exchange.fetch_order_book(asset))


In [ ]:
# Example balance
pprint(exchange.fetch_balance())


In [ ]:
#Example fetch order by id
pprint(exchange.fetch_order(19123522, asset.symbol))


In [ ]:
# Example fetch order
pprint(exchange.fetch_orders(asset))


In [ ]:
pprint(exchange.fetch_my_trades(asset, None, None))


In [ ]:
order_dct = {'amount': 10.0,
             'cost': 0.0,
             'datetime': '2018-01-17T05:49:23.119Z',
             'fee': None,
             'filled': 10.0,
             'id': '19123922',
             'price': 0.0,
             'remaining': 0.0,
             'side': 'buy',
             'status': 'closed',
             'symbol': 'XRP/BTC',
             'timestamp': 1516168163119,
             'type': 'market'}
order = Order.from_dict(order_dct)
pprint(order)

# Paper Exchange

In [ ]:
exchange = load_exchange(c.PAPER)

# Order manager does the is_balance_sufficient() check
# So we need to make sure currency exists when calling
# exchange directly
exchange.balance.add_currency(c.XRP)
exchange.fetch_balance()

## Limit Orders

In [ ]:
# LIMIT BUY (Pending - price lower than market so it doesn't fill)
asset = Asset(c.XRP, c.BTC)
order = exchange.create_limit_buy_order(asset, 20.0, price=0.0001)
order

In [ ]:
# Test Fetch
exchange.fetch_orders(asset)

In [ ]:
# LIMIT SELL (Pending - price higher than market so it doesn't fill)
asset = Asset(c.XRP, c.BTC)
order = exchange.create_limit_sell_order(asset, quantity=20., price=.0001)
order

In [ ]:
# Verify Balance
exchange.fetch_balance()

## Market Orders

In [ ]:
asset = Asset(c.XRP, c.BTC)
price = exchange.fetch_ticker(asset)['ask']
price

In [ ]:
# MARKET BUY
order = exchange.create_market_buy_order(asset, quantity=20.)
order

In [ ]:
# MARKET SELL
order = exchange.create_market_sell_order(asset, quantity=20.)
order

In [ ]:
# Verify Balance
exchange.fetch_balance()

# Order Manager

In [ ]:
exchange = load_exchange(c.PAPER)
asset = Asset(c.XRP, c.BTC)
exchange.fetch_balance()

## Limit Buy

In [ ]:
limit_buy = order_manager.build_limit_buy_order(exchange, asset, quantity=20., price=.0001)
limit_buy

In [ ]:
ex_order = order_manager.place_order(exchange, limit_buy)
ex_order

## Limit Sell

In [ ]:
limit_sell = order_manager.build_limit_sell_order(exchange, asset, quantity=20., price=.0001)
limit_sell

In [ ]:
ex_order = order_manager.place_order(exchange, limit_sell)
ex_order

## Market Buy

In [ ]:
market_buy = order_manager.build_market_buy_order(exchange, asset, quantity=20.)
market_buy

In [ ]:
ex_order = order_manager.place_order(exchange, market_buy)
ex_order

## Market Sell

In [ ]:
market_sell = order_manager.build_market_sell_order(exchange, asset, quantity=20.)
market_sell

In [ ]:
ex_order = order_manager.place_order(exchange, market_sell)
ex_order

## Known Exceptions

In [ ]:
# Insufficient funds 
# Right now we're returning a failed order, but the exchange doesn't so perhaps we want to catch
limit_buy = order_manager.build_limit_buy_order(exchange, asset, quantity=100000000., price=.0001)
ex_order = order_manager.place_order(exchange, limit_buy)
ex_order

# Trades

In [8]:
ex = load_exchange(c.BINANCE)
trade_dct = {'amount': 5.0,
  'cost': 0.000981,
  'datetime': '2018-01-04T22:20:38.781Z',
  'fee': {'cost': 0.005, 'currency': 'XRP'},
  'id': '6367895',
  'info': {'commission': '0.00500000',
           'commissionAsset': 'XRP',
           'id': 6367895,
           'isBestMatch': True,
           'isBuyer': True,
           'isMaker': False,
           'orderId': 13116770,
           'price': '0.00019620',
           'qty': '5.00000000',
           'time': 1515104437781},
  'order': '13116770',
  'price': 0.0001962,
  'side': 'buy',
  'symbol': 'XRP/BTC',
  'timestamp': 1515104437781,
  'type': None}
trade = ex._build_trade(trade_dct)
print("Trade", trade)

Trade {
    "id": "6367895",
    "exchange_id": "binance",
    "exchange_order_id": "13116770",
    "price": 0.0001962,
    "quantity": 5.0,
    "trade_time": "2018-01-04T22:20:38.781000+00:00",
    "fee": 0.005,
    "side": "BUY",
    "cost": 0.000981,
    "symbol": "XRP/BTC"
}


In [10]:
# testing fetching pub trade info
asset = Asset.from_symbol("ETH/BTC")
pub_trades = ex.fetch_public_trades(asset)
print(pub_trades)

[{
    "id": "22870978",
    "exchange_id": "binance",
    "exchange_order_id": null,
    "price": 0.091096,
    "quantity": 0.765,
    "trade_time": "2018-01-22T00:03:13.808000+00:00",
    "fee": null,
    "side": "BUY",
    "cost": 0.06968844,
    "symbol": "ETH/BTC"
}, {
    "id": "22870979",
    "exchange_id": "binance",
    "exchange_order_id": null,
    "price": 0.091095,
    "quantity": 0.022,
    "trade_time": "2018-01-22T00:03:13.520000+00:00",
    "fee": null,
    "side": "BUY",
    "cost": 0.0020040899999999996,
    "symbol": "ETH/BTC"
}, {
    "id": "22870980",
    "exchange_id": "binance",
    "exchange_order_id": null,
    "price": 0.091021,
    "quantity": 0.021,
    "trade_time": "2018-01-22T00:03:13.840000+00:00",
    "fee": null,
    "side": "SELL",
    "cost": 0.0019114410000000003,
    "symbol": "ETH/BTC"
}, {
    "id": "22870981",
    "exchange_id": "binance",
    "exchange_order_id": null,
    "price": 0.091021,
    "quantity": 1.099,
    "trade_time": "2018-01-22

In [11]:
# testing fetching my trade info
asset = Asset.from_symbol("XRP/BTC")
my_trades = ex.fetch_my_trades(asset)
print(my_trades)

[{
    "id": "6367895",
    "exchange_id": "binance",
    "exchange_order_id": "13116770",
    "price": 0.0001962,
    "quantity": 5.0,
    "trade_time": "2018-01-04T22:20:38.781000+00:00",
    "fee": 0.005,
    "side": "BUY",
    "cost": 0.000981,
    "symbol": "XRP/BTC"
}, {
    "id": "9779207",
    "exchange_id": "binance",
    "exchange_order_id": "19121540",
    "price": 0.00010221,
    "quantity": 20.0,
    "trade_time": "2018-01-17T05:40:20.753000+00:00",
    "fee": 0.02,
    "side": "BUY",
    "cost": 0.0020442,
    "symbol": "XRP/BTC"
}, {
    "id": "9780106",
    "exchange_id": "binance",
    "exchange_order_id": "19123522",
    "price": 0.00010254,
    "quantity": 20.0,
    "trade_time": "2018-01-17T05:47:43.192000+00:00",
    "fee": 2.05e-06,
    "side": "SELL",
    "cost": 0.0020508,
    "symbol": "XRP/BTC"
}, {
    "id": "9780329",
    "exchange_id": "binance",
    "exchange_order_id": "19123922",
    "price": 0.00010259,
    "quantity": 10.0,
    "trade_time": "2018-01-1

In [12]:
# testing _build_order
order = {'amount': 20.0,
 'cost': 0.0020442,
 'datetime': '2018-01-17T05:40:05.841Z',
 'fee': None,
 'filled': 20.0,
 'id': '19121540',
 'info': {'clientOrderId': 'XcRu3uVEd3fg4V657P6Tq2',
          'executedQty': '20.00000000',
          'icebergQty': '0.00000000',
          'isWorking': True,
          'orderId': 19121540,
          'origQty': '20.00000000',
          'price': '0.00010221',
          'side': 'BUY',
          'status': 'FILLED',
          'stopPrice': '0.00000000',
          'symbol': 'XRPBTC',
          'time': 1516167604841,
          'timeInForce': 'GTC',
          'type': 'LIMIT'},
 'price': 0.00010221,
 'remaining': 0.0,
 'side': 'buy',
 'status': 'closed',
 'symbol': 'XRP/BTC',
 'timestamp': 1516167604841,
 'type': 'limit'}

order = ex._build_orders([order])

ORDER Input {'amount': 20.0, 'cost': 0.0020442, 'datetime': '2018-01-17T05:40:05.841Z', 'fee': None, 'filled': 20.0, 'id': '19121540', 'info': {'clientOrderId': 'XcRu3uVEd3fg4V657P6Tq2', 'executedQty': '20.00000000', 'icebergQty': '0.00000000', 'isWorking': True, 'orderId': 19121540, 'origQty': '20.00000000', 'price': '0.00010221', 'side': 'BUY', 'status': 'FILLED', 'stopPrice': '0.00000000', 'symbol': 'XRPBTC', 'time': 1516167604841, 'timeInForce': 'GTC', 'type': 'LIMIT'}, 'price': 0.00010221, 'remaining': 0.0, 'side': 'buy', 'status': 'closed', 'symbol': 'XRP/BTC', 'timestamp': 1516167604841, 'type': 'limit'}
ORDER TRADES [{
    "id": "9779207",
    "exchange_id": "binance",
    "exchange_order_id": "19121540",
    "price": 0.00010221,
    "quantity": 20.0,
    "trade_time": "2018-01-17T05:40:20.753000+00:00",
    "fee": 0.02,
    "side": "BUY",
    "cost": 0.0020442,
    "symbol": "XRP/BTC"
}]
Order Output {
    "exchange_id": "binance",
    "price": 0.00010221,
    "status": "FILLE